### 설치코드

In [1]:
!pip install selenium
!pip install --upgrade selenium webdriver-manager


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: C:\Users\taeyoung\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [1]:
from webdriver_manager.chrome import ChromeDriverManager

driver_path = ChromeDriverManager().install()
print(driver_path)  # 크롬 드라이버 설치 경로 출력

C:\Users\taeyoung\.wdm\drivers\chromedriver\win64\128.0.6613.119\chromedriver-win32/THIRD_PARTY_NOTICES.chromedriver


## "THIRD_PARTY_NOTICES","chromedriver" ->(이건 내 경로임) 맨 뒤 알아서 경로 잘 넣을것 안되면 GPT ㄱㄱ 

### 테스트용

In [2]:
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# 크롬 드라이버 설치 경로
driver_path = ChromeDriverManager().install()

# 경로를 Service 객체에 전달
service = Service(driver_path.replace("THIRD_PARTY_NOTICES.chromedriver", "chromedriver.exe"))

# WebDriver 실행
driver = webdriver.Chrome(service=service)

# 원하는 페이지로 이동
driver.get('https://etk.srail.co.kr/cmc/01/selectLoginForm.do')

# 10초 대기
time.sleep(10)

# 필요한 작업을 수행한 후, 드라이버 종료
driver.quit()  # 모든 창을 닫고 드라이버 세션 종료

## 알람추가 실행용

In [41]:
import time
import winsound
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select

def reserve_train(member_num, pwd, depart_station, arrival_station, depart_date, depart_time, number_of_trains):
    """
    SRT 기차 예약 자동화 함수.
    
    Args:
        member_num (str): 회원번호
        pwd (str): 비밀번호
        depart_station (str): 출발역
        arrival_station (str): 도착역
        depart_date (str): 출발 날짜 (YYYYMMDD 형식)
        depart_time (str): 출발 시간 (00, 02, 04, ..., 20, 22 형식)
        number_of_trains (int): 검색 결과 상단에서 예약 가능 여부를 확인할 기차 수
    """
    
    # 크롬 드라이버 설치 경로
    driver_path = ChromeDriverManager().install()

    # 경로를 Service 객체에 전달
    service = Service(driver_path.replace("THIRD_PARTY_NOTICES.chromedriver", "chromedriver.exe"))

    # WebDriver 실행
    driver = webdriver.Chrome(service=service)

    try:
        # 원하는 페이지로 이동
        driver.get('https://etk.srail.co.kr/cmc/01/selectLoginForm.do')
        driver.implicitly_wait(15)

        # 로그인
        driver.find_element(By.ID, 'srchDvNm01').send_keys(member_num)  # 회원번호 입력
        driver.find_element(By.ID, 'hmpgPwdCphd01').send_keys(pwd)  # 비밀번호 입력
        driver.find_element(By.XPATH, '//*[@id="login-form"]/fieldset/div[1]/div[1]/div[2]/div/div[2]/input').click()
        driver.implicitly_wait(5)

        # 기차 조회 페이지로 이동
        driver.get('https://etk.srail.kr/hpg/hra/01/selectScheduleList.do')
        driver.implicitly_wait(5)

        # 출발지 입력
        dep_stn = driver.find_element(By.ID, 'dptRsStnCdNm')
        dep_stn.clear()
        dep_stn.send_keys(depart_station)

        # 도착지 입력
        arr_stn = driver.find_element(By.ID, 'arvRsStnCdNm')
        arr_stn.clear()
        arr_stn.send_keys(arrival_station)

        # 출발 날짜 설정
        elm_dptDt = driver.find_element(By.ID, "dptDt")
        driver.execute_script("arguments[0].setAttribute('style','display: True;')", elm_dptDt)
        Select(driver.find_element(By.ID, "dptDt")).select_by_value(depart_date)

        # 출발 시간 설정
        elm_dptTm = driver.find_element(By.ID, "dptTm")
        driver.execute_script("arguments[0].setAttribute('style','display: True;')", elm_dptTm)
        Select(driver.find_element(By.ID, "dptTm")).select_by_visible_text(depart_time)

        # 조회하기 버튼 클릭
        driver.find_element(By.XPATH, "//input[@value='조회하기']").click()
        driver.implicitly_wait(5)

        # 예약 가능한 기차 찾기
        reserved = False
        while True:
            for i in range(1, number_of_trains+1):
                # 기차 출발 및 도착 시간 정보 추출
                depart_time = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(4)").text
                arrival_time = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(5)").text
                train_time = f"{depart_time} ~ {arrival_time}"

                # 일반석 예약 상태 확인
                standard_seat = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(7)").text

                if "예약하기" in standard_seat:
                    # 예약 가능한 경우 해당 기차 정보 출력
                    train_info = driver.find_element(By.CSS_SELECTOR, f"#result-form > fieldset > div.tbl_wrap.th_thead > table > tbody > tr:nth-child({i}) > td:nth-child(5)").text
                    print(f"예약 가능: {train_time} (기차 정보: {train_info})")
                    
                    # 예약 진행
                    driver.find_element(By.XPATH, f"/html/body/div[1]/div[4]/div/div[3]/div[1]/form/fieldset/div[6]/table/tbody/tr[{i}]/td[7]/a/span").click()
                    reserved = True
                    break
                else:
                    print(f"매진: {train_time}")
            
            # 예약 성공 시 반복문 종료
            if reserved:
                break

            # 5초 기다리기 후 다시 조회
            time.sleep(5)
            submit = driver.find_element(By.XPATH, "//input[@value='조회하기']")
            driver.execute_script("arguments[0].click();", submit)
            print("새로고침")
            driver.implicitly_wait(10)
            time.sleep(3)

    finally:
        # 예약이 없을 때만 브라우저 종료
        if not reserved:
            driver.quit()

        # 예약이 완료된 후 5분 동안 알람 울리기
        if reserved:
            print("예약 완료, 알람 울림 시작.")
            for _ in range(5 * 60):  # 5분 동안 1초 간격으로 알람
                winsound.Beep(1000, 1000)  # 주파수 1000Hz, 1초 동안 울림
                time.sleep(3)  # 3초 대기


# 함수 호출 예시
reserve_train(
    member_num="회원번호",
    pwd="비밀번호",
    depart_station="수서",
    arrival_station="광주송정",
    depart_date="20240911",
    depart_time="10",               # 시간
    number_of_trains=4
)

매진: 수서
10:20 ~ 광주송정
12:18
매진: 수서
11:00 ~ 광주송정
12:42
매진: 수서
12:20 ~ 광주송정
14:03
매진: 수서
13:09 ~ 광주송정
14:55
새로고침
매진: 수서
10:20 ~ 광주송정
12:18
매진: 수서
11:00 ~ 광주송정
12:42
매진: 수서
12:20 ~ 광주송정
14:03
매진: 수서
13:09 ~ 광주송정
14:55


KeyboardInterrupt: 